In [2]:
# 安装必要的库（如果在Jupyter中运行，取消下面行的注释）
# !pip install pandas openpyxl python-docx docxtpl docx2pdf requests beautifulsoup4

import pandas as pd
import os
import platform
from docxtpl import DocxTemplate

def create_university_excel():
    """
    创建大学列表Excel文件 - 根据您提供的附件格式
    """
    print("开始创建大学列表Excel文件...")
    
    # 使用您提供的大学数据
    universities_data = {
        'Rank': [
            'Top30', '', '', '', '', '', '', '', '', '',
            'Top31~60', '', '', '', '', '', '', '', '', '',
            'Top61~90', '', '', '', '', '', '', '', '', ''
        ],
        'University Name': [
            'Harvard University',
            'Massachusetts Institute of Technology (MIT)',
            'University of California-Berkeley',
            'University of Chicago',
            'Paris School of Economics',
            'Princeton University',
            'Stanford University',
            'Yale University',
            'Toulouse School of Economics (TSE)',
            'Oxford University',
            'Columbia University',
            'Barcelona School of Economics (BSE)',
            'New York University (NYU)',
            'Brown University',
            'University of Pennsylvania',
            'London School of Economics (LSE)',
            'Boston University',
            'University College London (UCL)',
            'University of California-San Diego (UCSD)',
            'University of Michigan',
            'University of Queensland',
            'University of Virginia',
            'Georgetown University',
            'University of Minnesota',
            'New York University (NYU) - Stern School of Business',
            'University of California-Santa Barbara (UCSB)',
            'Washington University in St. Louis',
            'Pennsylvania State University',
            'University of Colorado',
            'University of California-Santa Cruz (UCSC)'
        ]
    }
    
    # 创建DataFrame
    df_universities = pd.DataFrame(universities_data)
    
    # 保存为Excel文件
    df_universities.to_excel('excellist1.xlsx', index=False)
    print("大学列表Excel文件已创建: excellist1.xlsx")
    
    return df_universities

def create_research_areas_excel():
    """
    创建研究领域、期刊和技能的Excel文件 - 根据您提供的附件格式
    """
    print("开始创建研究领域Excel文件...")
    
    # 使用您提供的研究领域数据
    research_areas_data = {
        'Research Area': ['Economics', 'Finance', 'Information Management'],
        'Top Journal 1': ['American Economic Review', 'Journal of Finance', 'MIS Quarterly'],
        'Top Journal 2': ['Econometrica', 'Journal of Financial Economics', 'Information Systems Research'],
        'Top Journal 3': ['Journal of Political Economy', 'Review of Financial Studies', 'Journal of the Association for Information Systems'],
        'Required Skill 1': ['Econometrics', 'Financial Modeling', 'Database Management'],
        'Required Skill 2': ['Data Analysis', 'Risk Management', 'Systems Analysis'],
        'Required Skill 3': ['Statistical Modeling', 'Investment Analysis', 'Project Management'],
        'Required Skill 4': ['Economic Theory', 'Financial Analysis', 'SQL'],
        'Required Skill 5': ['Policy Analysis', 'Python Programming', 'Business Process Optimization']
    }
    
    # 创建DataFrame
    df_research_areas = pd.DataFrame(research_areas_data)
    
    # 保存为Excel文件
    df_research_areas.to_excel('excellist2.xlsx', index=False)
    print("研究领域Excel文件已创建: excellist2.xlsx")
    
    return df_research_areas

def generate_application_letters():
    """
    生成申请信的主函数 - 修改为每所大学生成所有研究领域的申请信
    """
    # 创建输出目录
    output_dir = "HW_School_Application"
    os.makedirs(output_dir, exist_ok=True)
    
    print("开始读取Excel文件...")
    
    try:
        # 读取Excel文件
        df_universities = pd.read_excel('excellist1.xlsx')
        df_research_areas = pd.read_excel('excellist2.xlsx')
        
        print("Excel文件读取成功！")
        
    except FileNotFoundError as e:
        print(f"错误：找不到Excel文件 - {e}")
        print("将自动创建Excel文件...")
        df_universities = create_university_excel()
        df_research_areas = create_research_areas_excel()
    
    # 处理大学数据
    print("处理大学数据...")
    
    # 使用前向填充处理Rank列，确保每个大学都有正确的排名分类
    df_universities['Rank'] = df_universities['Rank'].ffill()
    
    # 获取各分组的大学
    universities_top30 = df_universities[df_universities['Rank'] == 'Top30']['University Name'].dropna().tolist()[:10]
    universities_top60 = df_universities[df_universities['Rank'] == 'Top31~60']['University Name'].dropna().tolist()[:10]
    universities_top90 = df_universities[df_universities['Rank'] == 'Top61~90']['University Name'].dropna().tolist()[:10]

    # 合并所有大学
    all_universities = universities_top30 + universities_top60 + universities_top90
    print(f"总共找到 {len(all_universities)} 所大学")

    # 准备研究领域数据
    print("处理研究领域数据...")
    research_areas_data = []
    for _, row in df_research_areas.iterrows():
        # 检查并处理可能的空值
        skills_dict = {}
        for i in range(1, 6):
            skill_col = f'Required Skill {i}'
            if skill_col in row and pd.notna(row[skill_col]):
                # 使用下划线替换空格，使变量名符合Jinja2语法
                skills_dict[f'Required_Skill_{i}'] = row[skill_col]
            else:
                skills_dict[f'Required_Skill_{i}'] = ""
        
        area_data = {
            'research_area': row['Research Area'],
            'journals': [
                row['Top Journal 1'], 
                row['Top Journal 2'], 
                row['Top Journal 3']
            ],
            'skills': skills_dict
        }
        research_areas_data.append(area_data)
    
    print(f"找到 {len(research_areas_data)} 个研究领域")

    # 职业目标映射
    career_goals = {
        'Economics': 'economic researcher',
        'Finance': 'financial analyst', 
        'Information Management': 'information systems manager'
    }

    print("开始生成申请信...")
    
    # 循环生成申请信 - 修改为每所大学对应所有研究领域
    generated_files = []
    application_count = 0
    
    # Windows系统PDF转换设置
    pdf_enabled = True
    try:
        from docx2pdf import convert
        print("PDF转换功能已启用")
    except ImportError:
        print("警告: 无法导入docx2pdf，将仅生成Word文档")
        pdf_enabled = False
    
    # 对每所大学，生成所有研究领域的申请信
    for i, university in enumerate(all_universities):
        for j, research_area_data in enumerate(research_areas_data):
            application_count += 1
            
            # 准备模板上下文 - 修复变量名，使用下划线代替空格
            context = {
                'research_area': research_area_data['research_area'],
                'university': university,
                'journals': research_area_data['journals'],
                'career_goal': career_goals.get(research_area_data['research_area'], 'researcher'),
                'Required_Skill_1': research_area_data['skills']['Required_Skill_1'],
                'Required_Skill_2': research_area_data['skills']['Required_Skill_2'],
                'Required_Skill_3': research_area_data['skills']['Required_Skill_3'],
                'Required_Skill_4': research_area_data['skills']['Required_Skill_4'],
                'Required_Skill_5': research_area_data['skills']['Required_Skill_5']
            }
            
            try:
                # 加载模板
                doc = DocxTemplate("MS word template.docx")
                
                # 渲染模板
                doc.render(context)
                
                # 生成文件名（移除特殊字符）
                safe_university_name = "".join(c for c in str(university) if c.isalnum() or c in (' ', '-', '_')).rstrip()
                safe_research_area = research_area_data['research_area'].replace(' ', '_')
                
                # 保存Word文档
                output_filename = f"Application_{application_count:03d}_{safe_university_name}_{safe_research_area}.docx"
                output_path = os.path.join(output_dir, output_filename)
                doc.save(output_path)
                
                generated_files.append(output_path)
                print(f"✓ 已生成: {output_filename}")
                
                # Windows系统PDF转换
                if pdf_enabled:
                    try:
                        pdf_output_path = output_path.replace('.docx', '.pdf')
                        convert(output_path, pdf_output_path)
                        generated_files.append(pdf_output_path)
                        print(f"  ↳ PDF已生成: {output_filename.replace('.docx', '.pdf')}")
                    except Exception as e:
                        print(f"  ⚠ PDF转换失败: {e}")
                else:
                    print(f"  ℹ PDF转换未启用")
                    
            except Exception as e:
                print(f"✗ 生成 {university} - {research_area_data['research_area']} 的申请信时出错: {e}")
                continue

    print(f"\n🎉 完成！")
    print(f"总共处理了 {len(all_universities)} 所大学 × {len(research_areas_data)} 个研究领域 = {application_count} 封申请信")
    print(f"生成的文件保存在 '{output_dir}' 目录中")
    
    # 显示生成的文件统计
    word_files = [f for f in generated_files if f.endswith('.docx')]
    pdf_files = [f for f in generated_files if f.endswith('.pdf')]
    
    print(f"\n📊 生成统计:")
    print(f"  - Word文档: {len(word_files)} 个")
    print(f"  - PDF文档: {len(pdf_files)} 个")
    print(f"  - 总计: {len(generated_files)} 个文件")
    
    return generated_files

# 主程序
if __name__ == "__main__":
    # 首先创建Excel文件
    print("=" * 50) 
    print("开始执行申请信生成程序")
    print("=" * 50)  
    
    # 创建Excel文件
    create_university_excel()
    create_research_areas_excel()
    
    print("\n" + "=" * 50)  
    print("开始生成申请信")
    print("=" * 50)  
    
    # 生成申请信
    generated_files = generate_application_letters()
    
    print("\n" + "=" * 50)  
    print("程序执行完毕")
    print("=" * 50)  

开始执行申请信生成程序
开始创建大学列表Excel文件...
大学列表Excel文件已创建: excellist1.xlsx
开始创建研究领域Excel文件...
研究领域Excel文件已创建: excellist2.xlsx

开始生成申请信
开始读取Excel文件...
Excel文件读取成功！
处理大学数据...
总共找到 30 所大学
处理研究领域数据...
找到 3 个研究领域
开始生成申请信...
PDF转换功能已启用
✓ 已生成: Application_001_Harvard University_Economics.docx


  0%|          | 0/1 [00:00<?, ?it/s]

  ↳ PDF已生成: Application_001_Harvard University_Economics.pdf
✓ 已生成: Application_002_Harvard University_Finance.docx


  0%|          | 0/1 [00:00<?, ?it/s]

  ⚠ PDF转换失败: Word.Application.Quit
✓ 已生成: Application_003_Harvard University_Information_Management.docx


...

  ⚠ PDF转换失败: Word.Application.Quit
✓ 已生成: Application_089_University of California-Santa Cruz UCSC_Finance.docx


  0%|          | 0/1 [00:00<?, ?it/s]

  ↳ PDF已生成: Application_089_University of California-Santa Cruz UCSC_Finance.pdf
✓ 已生成: Application_090_University of California-Santa Cruz UCSC_Information_Management.docx


  0%|          | 0/1 [00:00<?, ?it/s]

  ⚠ PDF转换失败: Word.Application.Quit

🎉 完成！
总共处理了 30 所大学 × 3 个研究领域 = 90 封申请信
生成的文件保存在 'HW_School_Application' 目录中

📊 生成统计:
  - Word文档: 90 个
  - PDF文档: 50 个
  - 总计: 140 个文件

程序执行完毕
